In [17]:
import json
import pandas as pd

In [18]:
with open("recipe-ingredient-nutrition-dataset-turkish-english-.json", "r", encoding="utf-8") as f:
    data = json.load(f)

recipes = data["Recipe"]
print(f"Toplam {len(recipes)} tarif bulundu.")

Toplam 685 tarif bulundu.


In [19]:
# Her tarif için gerekli alanları toplayalım
tarif_listesi = []

for tarif_id, tarif in recipes.items():
    tarif_listesi.append({
        "id": tarif_id,
        "ad": tarif.get("Name"),
        "kategori": tarif.get("MainCategory"),
        "malzemeler": tarif.get("IngridientNames"),
        "hazirlanis": tarif.get("RecipeDetails"),
        "aciklama": tarif.get("ShortDescription")
    })

# Pandas DataFrame oluştur
df = pd.DataFrame(tarif_listesi)

# İlk 5 satırı göster
df.head()


,id,ad,kategori,malzemeler,hazirlanis,aciklama
0,-MbNzUyCtfBnsghdACKO,Soslu Misket Köfte Tarifi,Köfte,dana kıyma;büyük boy soğan;sarımsak;yumurta;tu...,1.) Kıymayı geniş bir karıştırma kabına alın. ...,Enfes sosu ve tam kıvamında köfte harcıyla sos...
1,-MbNzVCQy3cTLIQ9G_ga,Fırında Kaşarlı Ispanak Yemeği Tarifi,Sebze,ıspanak (ayıklanmış ve temizlenmiş);patates;ze...,1.) Patateslerin kabuklarını soyup rendeleyin ...,Fırında kaşarlı ıspanak yemeği sebze tarifleri...
2,-MbNzVJVi59O2Z3pz8VJ,Kadınbudu Köfte Tarifi,Et,orta yağlı kıyma;yumurta;kuru soğan;sarımsak;h...,1.) 1/2 su bardağı pirinci tuz ve su ile birli...,"Köfte çeşitlerinden ismi ile müsemma, lezzetli..."
3,-MbNzVXQ61qgJsGTEeUh,Patatesli Bomba Köfte Tarifi,Köfte,orta yağlı dana kıyması;rendelenmiş soğan;mayd...,"1.) Köfteyi hazırlamak için, bir kasede kıyma,...",Patates ve köftenin o müthiş uyumu daha da pek...
4,-MbNzVkJJ0wL9fDAVnir,Sosyete Mantısı Tarifi,Hamur İşi,yufka;tereyağ;yumurta;kıyma;kuru soğan;tuz;kar...,1.) İç harcı için; bir kaba kıymayı alın ve üs...,"Mantı yapımını kolaylaştıran, yapımı çok prati..."


In [20]:
from transformers import pipeline

In [21]:
# HuggingFace üzerinden Türkçe BERT duygu analiz modeli yükle
sentiment_analyzer = pipeline("sentiment-analysis", model="savasy/bert-base-turkish-sentiment-cased")

In [22]:

# Kural tabanlı duygu tahmini
def duygu_tahmin_et(aciklama):
    if not isinstance(aciklama, str):
        return "belirsiz"
    aciklama = aciklama.lower()
    
    if any(word in aciklama for word in ["çikolata", "tatlı", "cheesecake", "pasta", "brownie", "kurabiye", "dondurma"]):
        return "mutlu eder"
    elif any(word in aciklama for word in ["çorba", "hafif", "sebze", "ıspanak", "haşlama", "kabağı", "yoğurtlu"]):
        return "rahatlatır"
    elif any(word in aciklama for word in ["baharatlı", "kızartma", "et", "köfte", "mangal", "tavuk", "acı", "ızgara"]):
        return "enerji verir"
    elif any(word in aciklama for word in ["pratik", "kolay", "hızlı", "az malzemeli", "tek tencerede"]):
        return "moral yükseltir"
    elif any(word in aciklama for word in ["sıcak", "anne yemeği", "ev yemeği", "nostalji", "çocukluğumuz"]):
        return "huzur verir"
    else:
        return "nötr"

# BERT tabanlı duygu tahmini
def nlp_label_sentiment(description):
    if pd.isna(description) or not description.strip():
        return "nötr"
    result = sentiment_analyzer(description)[0]
    label = result['label']
    if label == "Positive":
        return "olumlu"
    elif label == "Negative":
        return "olumsuz"
    return "nötr"

# Hibrit yaklaşım: her ikisini birlikte değerlendir
def hybrid_label_sentiment(description):
    rule_label = duygu_tahmin_et(description)
    if rule_label != "nötr":
        return rule_label
    
    nlp_label = nlp_label_sentiment(description)
    if nlp_label == "olumlu":
        return "mutlu eder"
    elif nlp_label == "olumsuz":
        return "rahatlatır"
    return "nötr"

# Veri çerçevesinde açıklamaya göre etiketleme
df["duygu_etiketi"] = df["aciklama"].apply(hybrid_label_sentiment)

# Sonuçları kontrol et
print(df[["ad", "aciklama", "duygu_etiketi"]].head())

# CSV olarak kaydet
df.to_csv("recipes_hybrid_labeled.csv", index=False, encoding="utf-8-sig")

print("Karma duygu etiketleme tamamlandı. Kaydedilen dosya: recipes_hybrid_labeled.csv")

                                      ad  \
0              Soslu Misket Köfte Tarifi   
1  Fırında Kaşarlı Ispanak Yemeği Tarifi   
2                 Kadınbudu Köfte Tarifi   
3           Patatesli Bomba Köfte Tarifi   
4                 Sosyete Mantısı Tarifi   

                                            aciklama duygu_etiketi  
0  Enfes sosu ve tam kıvamında köfte harcıyla sos...  enerji verir  
1  Fırında kaşarlı ıspanak yemeği sebze tarifleri...    rahatlatır  
2  Köfte çeşitlerinden ismi ile müsemma, lezzetli...  enerji verir  
3  Patates ve köftenin o müthiş uyumu daha da pek...  enerji verir  
4  Mantı yapımını kolaylaştıran, yapımı çok prati...  enerji verir  
Karma duygu etiketleme tamamlandı. Kaydedilen dosya: recipes_hybrid_labeled.csv


In [16]:
# Nötr olmayan var mı?
print(df["duygu_etiketi"].value_counts())

# Örnek birkaç açıklama gösterelim
df[["aciklama", "duygu_etiketi"]].sample(10)


duygu_etiketi
enerji verir       282
rahatlatır         152
mutlu eder         114
nötr                88
moral yükseltir     46
huzur verir          3
Name: count, dtype: int64


,aciklama,duygu_etiketi
74,Fırında falafel tarifiyle artık kalori derdine...,enerji verir
631,Kavala'nın aksine en lezzetlilerinin ülkemizde...,mutlu eder
205,Karadeniz'den sofralarımıza nefis bir tarif ge...,rahatlatır
682,Boşnak lezzeti olan begova çorbası soğuk haval...,rahatlatır
235,Hem çok kolay hem de çok lezzetli... Çaya dave...,enerji verir
608,Şerbetli tatlı gördüğünde yemeden duramayanlar...,mutlu eder
538,Antakya'nın muhteşem lezzetlerinden biri olan ...,enerji verir
113,"Köz patlıcan çorbası tarifiyle, közlenmiş patl...",rahatlatır
405,Yaz tadı kalamarı taze taze yemek için; onları...,enerji verir
207,"İster tatlı, ister tuzlu tüketebileceğiniz bu ...",mutlu eder
